In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os
import pandas as pd

load_dotenv()
hftoken = os.getenv("HUGGINGFACE_TOKEN")
login(hftoken)

In [11]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, concatenate_datasets, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


max_seq_length = 8192
dtype = None # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다. Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정됩니다.
load_in_4bit = True # 메모리 사용량을 줄이기 위해서는 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # gated model을 사용할 경우 허깅페이스 토큰을 입력해주시길 바라겠습니다.
)
# LoRA Adapter 선언
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # 0을 넘는 숫자를 선택하세요. 8, 16, 32, 64, 128이 추천됩니다.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # target module도 적절하게 조정할 수 있습니다.
    lora_alpha = 32,
    lora_dropout = 0, # 어떤 값이든 사용될 수 있지만, 0으로 최적화되어 있습니다.
    bias = "none",    # 어떤 값이든 사용될 수 있지만, "none"으로 최적화되어 있습니다.
    use_gradient_checkpointing = "unsloth", # 매우 긴 context에 대해 True 또는 "unsloth"를 사용하십시오.
    random_state = 42,
    use_rslora = False,
    loftq_config = None
)
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

model.print_trainable_parameters()

==((====))==  Unsloth 2024.10.3: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb. Max memory: 39.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM
Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32
trainable params: 1,412,956,160 || all params: 9,028,572,672 || trainable%: 15.6498


In [3]:
def formatting_prompts_func(examples):
    instructions = examples["question"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

In [5]:
dataset1 = load_dataset("llmflow/krx_dataset_by_gpt", split='train')
dataset2 = load_dataset("Linq-AI-Research/FinanceRAG",'FinQABench', split = "corpus")
dataset3 = load_dataset("amphora/rewrite-se-quant")['train']
dataset4 = load_dataset("amphora/krx-sample-instructions", split='train')
dataset5 = Dataset.from_pandas(pd.concat([
    pd.read_csv('금융시장_100q.csv'),
    pd.read_csv('재무회계_100q.csv'),
    pd.read_csv('재무회계_100q2.csv'),
    pd.read_csv('금융에이전트_100q.csv'),
    pd.read_csv('주가예측_100q.csv')
    ]))
dataset6 = Dataset.from_pandas(pd.read_csv('ep_info.csv'))
dataset7 = Dataset.from_pandas(pd.read_csv('rules.csv'))

tdf = pd.DataFrame(dataset2)
tdf = tdf[['title', 'text']]
tdf.columns = ['question', 'response']
dataset2 = Dataset.from_pandas(tdf)

tdf = pd.DataFrame(dataset3)
tdf = tdf[['query', 'output']]
tdf.columns = ['question', 'response']
dataset3 = Dataset.from_pandas(tdf)

tdf = pd.DataFrame(dataset4)
tdf = tdf[['prompt', 'response']]
tdf.columns = ['question', 'response']
dataset4 = Dataset.from_pandas(tdf)

tdf = pd.DataFrame(dataset6)
tdf = tdf[['Title', 'Context']]
tdf.columns = ['question', 'response']
dataset6 = Dataset.from_pandas(tdf)

tdf = pd.DataFrame(dataset7)
tdf = tdf[['Title', 'Context']]
tdf.columns = ['question', 'response']
dataset7 = Dataset.from_pandas(tdf)

dataset1 = dataset1.map(formatting_prompts_func, batched = True,)
dataset2 = dataset2.map(formatting_prompts_func, batched = True,)
dataset3 = dataset3.map(formatting_prompts_func, batched = True,)
dataset4 = dataset4.map(formatting_prompts_func, batched = True,)
dataset5 = dataset5.map(formatting_prompts_func, batched = True,)
dataset6 = dataset6.map(formatting_prompts_func, batched = True,)
dataset7 = dataset7.map(formatting_prompts_func, batched = True,)

# 데이터셋 통합
# combined_dataset = concatenate_datasets([dataset4, dataset5, dataset6, dataset7])
combined_dataset = dataset4

combined_dataset

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/21950 [00:00<?, ? examples/s]

Map:   0%|          | 0/25951 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4044 [00:00<?, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'response', 'formatted_text'],
    num_rows: 25951
})

In [7]:
pd.DataFrame(combined_dataset)['formatted_text'][0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.\n\n### Instruction:\n음악산업의 디지털화가 진행됨에 따라, 전통적인 음반 중심의 수익 모델에서 디지털 음악 서비스 중심으로 변화했음을 설명하는데, 이러한 변화가 이루어진 주된 원인은 무엇이며, 이에 따라 업계가 어떻게 대응하고 있는지 구체적인 예를 들어 설명하시오.\n\n### Response:\n음악산업의 디지털화 주된 원인은 인터넷과 мобиль 서비스의 발달로 인해 소비자들이 음악을 접근하고 소비하는 방식이 변화했기 때문입니다. 디지털 플랫폼의 등장으로 음원 스트리밍 서비스(예: 멜론, 스포티파이 등)가 보편화되면서, 소비자들은 과거의 음반 구매 대신 구독 기반의 서비스를 통해 음악에 쉽게 접근하게 되었습니다.\n\n이에 따라 음악업계는 다양한 대응 전략을 마련하고 있습니다. 예를 들어, 아티스트들은 음원 스트리밍 수익을 최적화하기 위해 SNS와 유튜브를 활용한 마케팅을 강화하고 있으며, 라이브 공연 및 팬미팅과 같은 오프라인 이벤트를 확대하여 추가 수익을 창출하고 있습니다. 또한, 일부 음악 레이블은 아티스트와의 협업을 통해 독점 콘텐츠를 제작하고, 메타버스와 같은 새로운 플랫폼을 이용해 가상 공연을 개최하는 등의 혁신적인 접근법을 시도하고 있습니다.<|im_end|>'

In [8]:
print(len(combined_dataset))

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # True로 설정하면 짧은 텍스트 데이터에 대해서는 더 빠른 학습 속도로를 보여줍니다.
    args = TrainingArguments( # TrainingArguments는 자신의 학습 환경과 기호에 따라 적절하게 설정하면 됩니다.
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        # num_train_epochs = 1,
        max_steps = 100,
        learning_rate = 1e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_format.format(
        """다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.
### 질문: 다음 중 우리나라 주식시장 매매 제도에 대한 기술로 맞는 것은?
### 선택지: 
A. 개장 시간은 오전 10시다.
B. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.
C. 코스닥시장에는 가격 제한폭이 없다.
D. 점심시간(12~1시)에는 휴장한다.
E. 동시호가는 폐장 시간에만 적용한다.
F. K-OTC시장의 가격 제한폭은 전일 종가 대비 상하 30%이다.
G. K-OTC시장의 운영시간은 09:00부터 16:00까지이다.""", # instruction
        "", # output 생성을 위해 빈 칸으로 설정
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs))
for item in tokenizer.batch_decode(outputs):
    print(item)

25951


Map (num_proc=4):   0%|          | 0/25951 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n아래는 지시사항과 추가 정보가 포함된 내용입니다. 주어진 내용을 참고하여 한국어로 적절하게 답변해 주세요.\n\n### Instruction:\n다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문: 다음 중 우리나라 주식시장 매매 제도에 대한 기술로 맞는 것은?\n### 선택지: \nA. 개장 시간은 오전 10시다.\nB. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.\nC. 코스닥시장에는 가격 제한폭이 없다.\nD. 점심시간(12~1시)에는 휴장한다.\nE. 동시호가는 폐장 시간에만 적용한다.\nF. K-OTC시장의 가격 제한폭은 전일 종가 대비 상하 30%이다.\nG. K-OTC시장의 운영시간은 09:00부터 16:00까지이다.\n\n### Response:\n본 질문의 정답을 찾기 위해 각 선택지들을 살펴봅니다.\n\nA. 개장 시간은 오전 10시다.\n이 문장은 주식 시장의 개장 시간에 대한 정보이며, 일반적으로 주식 시장의 개장 시간은 이와 같으므로 올바른 정보라고 판단됩니다.\n\nB. 유가증권시장의 가격 제한폭은 전일 종가 대비 상하 15%이다.\n이 문장은 주식 시장의 가격 제한폭에 대한 정보이며, 유가증권시장의 가격 제한폭이 전일 종가 대비 상하 15%라는 정보는 일반적으로 주식 시장의 규칙에 따른 정보라고 판단됩니다.\n\nC. 코스닥시장에는 가격 제한폭이 없다.\n코스닥시장에 대한 정보입니다. 실제로 코스닥시장에는 가격 제한폭이 있으므로 이 문장은 잘못된 정보라고 판단됩니다.\n\nD. 점심시간(12~1시)에는 휴장한다.\n이 문장은 주식 시장의 휴장 시간에 대한 정보이며, 대부분의 주식 시장에서 점심시간에는 휴장하지 않는다

In [9]:
# LoRA Adapter 저장
# model.save_pretrained("lora_model")
# tokenizer.save_pretrained("lora_model")

# # Merged model 저장 및 업로드
# model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("llmflow/krx_Qwen2_7B_Instruct_v2", tokenizer, save_method = "merged_16bit", token = hftoken) # 개인 huggingface token을 사용하여 업로드할 수 있습니다.

Unsloth: You are pushing to hub, but you passed your HF username = llmflow.
We shall truncate llmflow/krx_Qwen2_7B_Instruct_v2 to krx_Qwen2_7B_Instruct_v2


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 29.79 out of 48.0 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 20.48it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/llmflow/krx_Qwen2_7B_Instruct_v2
